In [1]:
from nltk.corpus import wordnet as wn
import nltk

In [2]:
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [3]:
# Hyperparameters

#concreteness_threshold = 2
concreteness_threshold = 7

# Number of positive examples we want for each classifier
#pos_count = 3
#neg_count = 9
#pos_count = 10
#neg_count = 20
pos_count = 15
neg_count = 40


test_pct = 0.33 #0.33

embedding_file = 'ddata/clip.bertvocab.embeddings.513.txt'
embedding_pickle = 'ddata/clip.bertvocab.embeddings.513.pkl'
concreteness_file = 'ddata/AC_ratings_google3m_koeper_SiW.csv'
concreteness_pickle = 'ddata/AC_ratings_google3m_koeper_SiW.pkl'

In [4]:
# wac2vec

if os.path.isfile(embedding_pickle):
    wac2vec = pickle.load(open(embedding_pickle, 'rb'))
else:
    with open(embedding_file) as f:
        f = f.readlines()
        f = [line.split() for line in f]
        wac2vec = {line[0]:np.array(line[1:], dtype=np.float32) for line in f}
        pickle.dump(wac2vec, open(embedding_pickle, "wb" ))

len(wac2vec)

30522

In [5]:
# concr_scores

if os.path.isfile(concreteness_pickle):
    concr_scores = pickle.load(open(concreteness_pickle, 'rb'))
else:
    concr_scores = pd.read_csv(concreteness_file, delimiter='\t')
    concr_scores = concr_scores.dropna()
    concr_scores.WORD = concr_scores.WORD.apply(lambda x: x.replace("_", " "))
    concr_scores.index = concr_scores.WORD
    pickle.dump(concr_scores, open(concreteness_pickle, "wb" ))

len(concr_scores)

2168990

In [6]:
X_features = []
for word in list(wac2vec.keys())[7000:7600]:
    X_features.append(wac2vec[word])

In [7]:
# Visualize
pca = PCA()
pca.fit_transform(X_features)
pca_variance = pca.explained_variance_

In [10]:
%load_ext autoreload

In [11]:
%autoreload 2

In [80]:
from concreteabstract import ConcreteAbstract
ca = ConcreteAbstract(wac2vec, concr_scores, wn)
ca.init_abstraction_tree(min_rating=9)
ca.grow_abstraction_tree()

  0%|          | 0/14510 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [81]:
#ca.build_display_tree(wn.synset('entity.n.01'), char_limit=5)

In [83]:
ca.add_pos_neg_all()

  0%|          | 0/4 [00:00<?, ?it/s]

In [84]:
ca.fill_out_train_test()

  0%|          | 0/4 [00:00<?, ?it/s]

In [85]:
ca.build_classifiers()

  0%|          | 0/4 [00:00<?, ?it/s]

In [86]:
ca.comp_random_baseline()

0.5

In [87]:
ca.comp_most_common_baseline()

0.5945945945945946

In [88]:
ca.evaluate_vs_negative_examples()

0.625

In [89]:
score, rand_baseline = ca.evaluate_vs_distractors(10)
print("score:    ", score)
print("baseline: ", rand_baseline)

  0%|          | 0/4 [00:00<?, ?it/s]

score:     0.8
baseline:  0.5


In [90]:
ca = ConcreteAbstract(wac2vec, concr_scores, wn, pos_count=10, neg_count=20)

In [95]:
ca.build_all(min_rating=9, verbose=False)

UnboundLocalError: local variable 'tqdm' referenced before assignment